In [1]:
import numpy as np 
import glob
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.models import *
from keras.layers.merge import concatenate
from keras.layers import Input, merge, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Using TensorFlow backend.


# Data processing

In [2]:
# make config
rows = 256
cols = 256
train_path = "train"
label_path = "label"
img_type = "jpg"
test_path = "test"
npy_path = "npydata"

In [3]:
def create_data(data_path, npy_path, rows, cols):   
    imgs = glob.glob("data\\" + data_path + "\\*." + img_type)
    i = 0
    img_count = len(imgs)
    imgs_data = np.ndarray((img_count, rows, cols, 1), dtype=np.uint8)
    for img_full_name in imgs:
        img_name = img_full_name[img_full_name.rindex("\\") + 1:]
        # test
        img = load_img("data\\" + data_path + "\\" + img_name, grayscale = True)
        img = img_to_array(img)
        imgs_data[i] = img
        if i % 10 == 0:
            print('Done: {0}/{1} images'.format(i, img_count))
        i += 1
    print('Created.')
    np.save("data\\" + npy_path + '\\imgs_' + data_path + '.npy', imgs_data)
    print('Saving to .npy files done.')

In [4]:
def load_data(data_path, npy_path, process_colors = False):
    imgs = np.load("data\\" + npy_path + '\\imgs_' + data_path + '.npy')
    imgs = imgs.astype('float32')
    imgs /= 255
    if (process_colors):
        imgs[imgs > 0.5] = 1
        imgs[imgs <= 0.5] = 0
    return imgs

# U-net

In [5]:
def bottleneck1(rows, cols, k_size = 3):

    inputs = Input((rows, cols, 1))
    merge_axis = -1
    conv1 = Conv2D(filters=32, kernel_size=k_size, padding='same', activation='relu')(inputs)
    conv1 = Conv2D(filters=32, kernel_size=k_size, padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(pool1)
    conv2 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(pool2)
    conv3 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    output = pool3
    model = Model(inputs, output)
    return model

In [6]:
def bottleneck2(rows, cols, k_size = 3):
    
    inputs = Input((32, 32, 64))
    merge_axis = -1
    conv4 = Conv2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(inputs)
    conv4 = Conv2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(pool4)

    up1 = UpSampling2D(size=(2, 2))(conv5)
    conv6 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(up1)
    conv6 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(conv6)
    merged1 = concatenate([conv4, conv6], axis=merge_axis)
    conv6 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(merged1)

    up2 = UpSampling2D(size=(2, 2))(conv6)
    conv7 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(up2)
    conv7 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(conv7)
    #merged2 = concatenate([conv3, conv7], axis=merge_axis)
    conv7 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(conv7)

    up3 = UpSampling2D(size=(2, 2))(conv7)
    conv8 = Conv2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(up3)
    conv8 = Conv2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(conv8)
    #merged3 = concatenate([conv2, conv8], axis=merge_axis)
    conv8 = Conv2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(conv8)

    up4 = UpSampling2D(size=(2, 2))(conv8)
    conv9 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(up4)
    conv9 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv9)
    #merged4 = concatenate([conv1, conv9], axis=merge_axis)
    conv9 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv9)

    conv10 = Conv2D(filters=1, kernel_size=k_size, padding='same', activation='sigmoid')(conv9)

    output = conv10
    model = Model(inputs, output)
    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [7]:
def train(train_path, test_path, label_path, npy_path, rows, cols):
    imgs_train = load_data(train_path, npy_path)
    imgs_test = load_data(test_path, npy_path)
    imgs_mask_train = load_data(label_path, npy_path, True)
    model = get_model(rows, cols)

    model_checkpoint = ModelCheckpoint('unet.hdf5', monitor='loss',verbose=1, save_best_only=True)
    model.fit(imgs_train, imgs_mask_train, batch_size=4, nb_epoch=5, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

    imgs_mask_test = model.predict(imgs_test, batch_size=1, verbose=1)
    save_results(imgs_mask_test)

In [41]:
def predict(test_path, npy_path, rows, cols):
    imgs_test = load_data(test_path, npy_path)
    model = build_UNet2D_4L(rows, cols)
    model.load_weights('unet2.hdf5')
    imgs_mask_test = model.predict(imgs_test, batch_size = 4, verbose=1)
    save_results(imgs_mask_test)

In [43]:
def test(train_path, test_path, label_path, npy_path, rows, cols):
    imgs_train = load_data(train_path, npy_path)
    imgs_test = load_data(test_path, npy_path)
    imgs_mask_train = load_data(label_path, npy_path, True)
    
    model = build_UNet2D_4L(rows, cols)
    model.load_weights('trained_model.hdf5')
    weights_list = model.get_weights()
    
    model_checkpoint = ModelCheckpoint('unet2.hdf5', monitor='loss',verbose=1, save_best_only=True)
    model.fit(imgs_train, imgs_mask_train, batch_size=4, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])
    
    imgs_mask_test = model.predict(imgs_test, batch_size = 4, verbose=1)
    save_results(imgs_mask_test)   

In [35]:
def build_UNet2D_4L(rows, cols, k_size=3):
    inputs = Input((rows, cols, 1))
    merge_axis = -1 # Feature maps are concatenated along last axis (for tf backend)
    conv1 = Conv2D(filters=32, kernel_size=k_size, padding='same', activation='relu', trainable=False)(inputs)
    conv1 = Conv2D(filters=32, kernel_size=k_size, padding='same', activation='relu', trainable=False)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu', trainable=False)(pool1)
    conv2 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu', trainable=False)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    """""" """""" """""" """""" """""" """""""""""" """""" """""" """""" """""" """""""""""" """""" """""" """""" """""" """"""

    conv3 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(pool2)
    conv3 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    """""" """""" """""" """""" """""" """""""""""" """""" """""" """""" """""" """"""

    conv4 = Conv2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(pool3)
    conv4 = Conv2D(filters=128, kernel_size=k_size, padding='same', activation='relu')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(pool4)

    up1 = UpSampling2D(size=(2, 2))(conv5)
    conv6 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(up1)
    conv6 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(conv6)
    merged1 = concatenate([conv4, conv6], axis=merge_axis)
    conv6 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(merged1)
    
    """""" """""" """""" """""" """""" """""""""""" """""" """""" """""" """""" """"""

    up2 = UpSampling2D(size=(2, 2))(conv6)
    conv7 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(up2)
    conv7 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(conv7)
    merged2 = concatenate([conv3, conv7], axis=merge_axis)
    conv7 = Conv2D(filters=256, kernel_size=k_size, padding='same', activation='relu')(merged2)
    
    """""" """""" """""" """""" """""" """""""""""" """""" """""" """""" """""" """""""""""" """""" """""" """""" """""" """"""

    up3 = UpSampling2D(size=(2, 2))(conv7)
    conv8 = Conv2D(filters=128, kernel_size=k_size, padding='same', activation='relu', trainable=False)(up3)
    conv8 = Conv2D(filters=128, kernel_size=k_size, padding='same', activation='relu', trainable=False)(conv8)
    merged3 = concatenate([conv2, conv8], axis=merge_axis)
    conv8 = Conv2D(filters=128, kernel_size=k_size, padding='same', activation='relu', trainable=False)(merged3)

    up4 = UpSampling2D(size=(2, 2))(conv8)
    conv9 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu', trainable=False)(up4)
    conv9 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu', trainable=False)(conv9)
    merged4 = concatenate([conv1, conv9], axis=merge_axis)
    conv9 = Conv2D(filters=64, kernel_size=k_size, padding='same', activation='relu', trainable=False)(merged4)

    conv10 = Conv2D(filters=1, kernel_size=k_size, padding='same', activation='sigmoid', trainable=False)(conv9)

    output = conv10
    model = Model(inputs, output)
    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
test(train_path, test_path, label_path, npy_path, rows, cols)

C:\Users\artem.eprev\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # This is added back by InteractiveShellApp.init_path()


Train on 115 samples, validate on 29 samples
Epoch 1/10
115/115 [==============================] - 303s 3s/step - loss: 0.5979 - acc: 0.8660 - val_loss: 0.1003 - val_acc: 0.9672

Epoch 00001: loss improved from inf to 0.59790, saving model to unet2.hdf5
Epoch 2/10
115/115 [==============================] - 300s 3s/step - loss: 0.2681 - acc: 0.9123 - val_loss: 0.0839 - val_acc: 0.9705

Epoch 00002: loss improved from 0.59790 to 0.26814, saving model to unet2.hdf5
Epoch 3/10
115/115 [==============================] - 292s 3s/step - loss: 0.1483 - acc: 0.9454 - val_loss: 0.0871 - val_acc: 0.9699

Epoch 00003: loss improved from 0.26814 to 0.14827, saving model to unet2.hdf5
Epoch 4/10
115/115 [==============================] - 289s 3s/step - loss: 0.1143 - acc: 0.9582 - val_loss: 0.0891 - val_acc: 0.9705

Epoch 00004: loss improved from 0.14827 to 0.11429, saving model to unet2.hdf5
Epoch 5/10
115/115 [==============================] - 288s 3s/step - loss: 0.0959 - acc: 0.9643 - val_loss:

In [37]:
def save_results(imgs):
    i = 1
    for img in imgs:
            img = array_to_img(img)
            img.save("data\\results\\%d.jpg"%(i))   
            i += 1

# Main

In [39]:
create_data(train_path, npy_path, rows, cols)
create_data(label_path, npy_path, rows, cols)
create_data(test_path, npy_path, rows, cols)

Done: 0/144 images
Done: 10/144 images
Done: 20/144 images
Done: 30/144 images
Done: 40/144 images
Done: 50/144 images
Done: 60/144 images
Done: 70/144 images
Done: 80/144 images
Done: 90/144 images
Done: 100/144 images
Done: 110/144 images
Done: 120/144 images
Done: 130/144 images
Done: 140/144 images
Created.
Saving to .npy files done.
Done: 0/144 images
Done: 10/144 images
Done: 20/144 images
Done: 30/144 images
Done: 40/144 images
Done: 50/144 images
Done: 60/144 images
Done: 70/144 images
Done: 80/144 images
Done: 90/144 images
Done: 100/144 images
Done: 110/144 images
Done: 120/144 images
Done: 130/144 images
Done: 140/144 images
Created.
Saving to .npy files done.
Done: 0/5 images
Created.
Saving to .npy files done.


In [ ]:
train(train_path, test_path, label_path, npy_path, rows, cols)

In [42]:
predict(test_path, npy_path, rows, cols)

5/5 [==============================] - 5s 1s/step
